In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
import numpy as np
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import time

In [2]:
data = pd.read_csv('data/clean_data.csv')

In [3]:
data.head()

,is_offensive,text
0,0,Then go to the village pump and suggest they c...
1,1,ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...
2,1,Dis hoe wasnt dis violent on Lottery Ticket 😂😂
3,0,It is better for Atabay not helping the banned...
4,0,"""is in CamelCase. """"SiCKO"""" is not CamelCase,..."


In [4]:
texts = data['text'].astype(str)
y = data['is_offensive'].tolist()

In [5]:
st = time.time()
# Vectorize the text
vectorizer = CountVectorizer(stop_words='english', min_df=0.0001)
X = vectorizer.fit_transform(texts)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Train the model
model = LinearSVC(class_weight="balanced", dual=False, tol=1e-2, max_iter=1e5)
cclf = CalibratedClassifierCV(base_estimator=model)
cclf.fit(X_train, y_train)
print(time.time() - st)

/home/kunal/anaconda3/envs/pytorchenv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


271.28612995147705


In [7]:
hits = 0
total = X_train.shape[0]
for i in range(total):
    temp = cclf.predict_proba(X_train[i])
    if temp[0].argmax() == y_train[i]:
        hits += 1
print("Train Accuracy: ", (hits/total)*100)

Train Accuracy:  96.15256456949794


In [6]:
hits = 0
total = X_test.shape[0]
for i in range(total):
    temp = cclf.predict_proba(X_test[i])
    if temp[0].argmax() == y_test[i]:
        hits += 1
print("Test Accuracy: ", (hits/total)*100)

Test Accuracy:  95.28992713399751


In [22]:
for no in range(0, 4001, 500):
    st = time.time()
    for i in range(no):
        temp = cclf.predict_proba(X_test[i])
        if temp[0].argmax() == y_test[i]:
            hits += 1
    print(time.time() - st, no)        

3.5762786865234375e-06 0
0.30098986625671387 500
0.5698459148406982 1000
0.8834776878356934 1500
1.1732404232025146 2000
1.4700093269348145 2500
1.75529146194458 3000
2.1890463829040527 3500
2.721097707748413 4000
